# Import

In [33]:
import pandas as pd
import numpy as np

# Data

In [2]:
order_products__train = pd.read_csv("../instacart_2017_05_01/order_products__train.csv")
order_products__prior = pd.read_csv("../instacart_2017_05_01/order_products__prior.csv")
products = pd.read_csv("../instacart_2017_05_01/products.csv")
orders = pd.read_csv("../instacart_2017_05_01/orders.csv")

# Train data

In [3]:
orders_train = orders[orders['eval_set'] == 'train'].reset_index()
orders_train = orders_train[['user_id', 'order_id']]

In [4]:
order_products__train = order_products__train[['order_id', 'product_id', 'reordered']]
order_products__train = order_products__train.groupby('order_id')['product_id'].apply(list).reset_index()

In [5]:
user_products__train = pd.merge(order_products__train, orders_train, on='order_id')
user_products__train = user_products__train[['user_id', 'product_id']]

In [6]:
user_products__train.set_index('user_id', inplace=True)
user_products__train.head()

,product_id
user_id,
112108,"[49302, 11109, 10246, 49683, 43633, 13176, 472..."
79431,"[39612, 19660, 49235, 43086, 46620, 34497, 486..."
42756,"[11913, 18159, 4461, 21616, 23622, 32433, 2884..."
17227,"[20574, 30391, 40706, 25610, 27966, 24489, 39275]"
56463,"[8859, 19731, 43654, 13176, 4357, 37664, 34065..."


# Top products

In [8]:
ordered_products = order_products__prior.merge(orders, on='order_id', how='inner')
ordered_products.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,prior,3,5,9,8.0
1,2,28985,2,1,202279,prior,3,5,9,8.0
2,2,9327,3,0,202279,prior,3,5,9,8.0
3,2,45918,4,1,202279,prior,3,5,9,8.0
4,2,30035,5,0,202279,prior,3,5,9,8.0


In [17]:
popular_items = ordered_products['product_id'].value_counts()
top_10_items = list(popular_items.iloc[:10].index)

In [18]:
# Recommener function will output product_ids. Use this dict to then get human names for food
product_id_to_name_dict = pd.Series(products['product_name'].values, index=products['product_id']).to_dict()

def product_id_to_name(product_ids):
    return [product_id_to_name_dict[id] for id in product_ids]

In [19]:
product_id_to_name(top_10_items)

['Banana',
 'Bag of Organic Bananas',
 'Organic Strawberries',
 'Organic Baby Spinach',
 'Organic Hass Avocado',
 'Organic Avocado',
 'Large Lemon',
 'Strawberries',
 'Limes',
 'Organic Whole Milk']

# "Model" evaluator

In [24]:
user_ids = user_products__train.index

In [31]:
def get_model_accuracy(user_ids, item_list):
    accuracies = []
    for user_id in user_ids:
        recommended_items = set(item_list)
        real_items = set(user_products__train.loc[user_id][0])
        if len(real_items) != 0:
            accuracy = len(recommended_items.intersection(real_items)) / len(real_items)
            accuracies.append(accuracy)
    model_accuracy = np.mean(accuracies)
    return model_accuracy

In [34]:
top_10_accuracy = get_model_accuracy(user_ids, top_10_items)

In [35]:
top_10_accuracy

0.06984315126267857

In [38]:
.23 / 0.07

3.2857142857142856

In [39]:
7 * 3.29

23.03